# Übung: Maximum Weighted Matching

Die Beratungsfirma *UConsult* will Projektleiter auf Projekte verteilen, sodass der Gewinn maximal ist. Dabei kann jeder Projektleiter nur einem Projekt zugeordnet werden und jedes Projekt benötigt einen Projektleiter. Der Gewinn eines Projekts hängt davon ab, welcher Projektleiter ihm zugeordnet wird. Projekte, die keinen Projektleiter haben, generieren keinen Gewinn.

Zunächst werden benötigte Softwarepakete importiert.

In [ ]:
import numpy as np  ## https://numpy.org Python-Bibliothek für wissenschaftliches Rechnen 
import random
import time

## Die Instanz

Die Eingabe ist eine Matrix $W$, in der es für jeden Mitarbeiter $a$ eine Zeile gibt und für jedes Projekt $b$ eine Spalte. Der Eintrag $W_{a,b}$ enthält den Gewinn, den man erzielen würde, wenn man Mitarbeiter $a$ dem Projekt $b$ zuordnet. Mit der Methode <code>random_instance</code> wird eine solche Profitmatrix erzeugt. 

In [ ]:
def random_instance(left_size,right_size,edge_proba,seed=-1):
    """
    generates random instances with profits from 1 to 100 (or 0 if row/column combination is not feasible).
    left_size, right_size: number of rows and columns in profit matrix
    edge_proba: probability that some row/column combination has positive profit.
    """
    if seed>0:
        random.seed(seed)
    profits=np.zeros((left_size,right_size))  # initialise profits to 0
    for l in range(left_size):
        for r in range(right_size):
            if random.random()<=edge_proba:   # do random experiment to see whether row/column feasible
                profits[l,r]=random.randint(1,100)  # if yes, draw random profit
    return profits

profits=random_instance(4,5,0.5,seed=27)
profits

Wie sieht nun eine Zuteilung von Berater:innen auf Projekte aus? Wir kodieren das als Liste von Paaren <code>(berater,projekt)</code>. Dh die Liste <code>[(0,4),(1,3),(2,2)]</code> bedeutet, dass Beraterin 0 auf Projekt 4 eingesetzt wird, Beraterin 1 auf Projekt 3 und Beraterin 2 auf Projekt 2. Wir stellen auch eine Funktion bereit, um den Gesamtprofit zu berechnen.

In [ ]:
def compute_profit(assignment,profits):
    return sum([profits[c,p] for c,p in assignment])

assignment=[(0,4),(1,3),(2,2)]
compute_profit(assignment,profits)

## Greedy-Algorithmus

Der greedy-Algorithmus ist denkbar einfach: in jeder Runde wählen wir unter den Zeile/Spalten-Kombinationen, die noch möglich sind, diejenige mit höchstem Profit aus. Um nicht in jeder Runde neu nach der Kombination mit höchstem Profit suchen zu müssen, werden die Zeile/Spalten-Paare einmal am Anfang nach Profit sortiert. Schließlich muss nur noch gewährleistet werden, dass wir uns die Zeilen und Spalten merken, die bereits benutzt wurden.

In [ ]:
def greedy_max_matching(profits):
    """
    expects profit matrix (numpy array) as input, all entries should be non-negative
    outputs: total profit, assignment
    where assignment is a list of pairs (i,j) meaning that row i is matched with column j
    """
    total_profit=0
    assignment=[]
    L,R=profits.shape  # L-> number of rows, R-> number of columns
    used_left,used_right=[],[]  # keep track of which rows/columns are already matched
    potential_profits=[(profits[l,r],l,r) for l in range(L) for r in range(R) if profits[l,r]>0]
    potential_profits.sort(reverse=True)  # sort row/column pairs by profit, highest first
    for profit,l,r in potential_profits:
        if not l in used_left and not r in used_right:  # if row/column still feasible, take it
            used_left.append(l)     # row becomes infeasible
            used_right.append(r)    # column becomes infeasible
            assignment.append((l,r)) # keep track of assignment
            total_profit+=profit    # keep track of profit
    return total_profit,assignment

profits=random_instance(10,12,0.3,seed=42)
profit,assignment=greedy_max_matching(profits)
print("Profit: {}k€".format(profit))

## Zufallssuche

Als nächstes wollen wir eine Zufallssuche implementieren. Als Hilfsfunktion ist hier eine Methode, die eine zufällige Verteilung der Berater:innen auf die Projekte realisiert. 

In [ ]:
def create_random_assignment(profits):
    num_consultants,num_projects=profits.shape
    projects=list(range(num_projects))
    consultants=list(range(num_consultants))
    random.shuffle(projects)
    random.shuffle(consultants)
    return [(c,p) for c,p in zip(consultants,projects)]

create_random_assignment(profits)

### Aufgabe: Zufallssuche
Implementieren Sie die Methode <code>random_search(profits,time_budget)</code>! Erzeugen Sie dafür mit Hilfe von <code>create_random_assignment</code> zufällige Zuteilungen und geben Sie die beste aus, die innerhalb der gegebenen Zeit <code>time_budget</code> (in Sekunden) gefunden wird.

In [ ]:
def random_search(profits,time_budget):
    ### Ihr Code hier ###

    ### Ende Ihres Codes ###
    return best_profit,best_assignment
    
profits=random_instance(10,12,0.3,seed=42)
profit,assignment=random_search(profits,2)
print("Realisierter Profit: {}k€".format(profit))

## Hill climbing

Wir wollen die Lösungen noch ein wenig mit *hill climbing* verbessern. Es stellt sich dabei heraus, dass wir die Zuteilung besser ein bisschen anders darstellen: Und zwar werden wir direkt die Zuteilung von Berater:innen auf Projekte benötigen -- und auch umgekehrt von Projekten auf Berater:innen. Dh, wenn <code>[(0,4),(3,1),(2,2)]</code> die Zuteilung ist, dann wollen wir eine Liste <code>consultants_mapping</code> der Form <code>[4,None,2,1]</code>. An der Liste können wir direkt ablesen, dass Beraterin 0 auf Projekt 4 eingesetzt wird, Beraterin 1 keinem Projekt zugeordnet ist usw. Gleichzeitig wird sich die Liste <code>projects_mapping</code> als nützlich erweisen, in diesem Fall wäre das <code>[None,3,2,None,0]</code>. Dh, wir sehen, dass keine Beraterin Projekt 0 zugeordnet ist, Projekt 1 von Beraterin 3 betreut wird usw. Die Methode <code>get_mappings</code> berechnet diese Listen.

In [ ]:
def get_mappings(assignment,profits):
    num_consultants=profits.shape[0]
    num_projects=profits.shape[1]
    consultants_mapping=[None]*num_consultants
    projects_mapping=[None]*num_projects
    for consultant,project in assignment:
        consultants_mapping[consultant]=project
        projects_mapping[project]=consultant
    return consultants_mapping,projects_mapping

profits=random_instance(4,5,0.3,seed=42)
assignment=[(0,4),(3,1),(2,2)]
consultants_mapping,projects_mapping=get_mappings(assignment,profits)
consultants_mapping,projects_mapping

Wir haben fehlende Zuteilungen mit <code>None</code> kodiert. Wie überprüft man auf <code>None</code>? So:

In [ ]:
if consultants_mapping[2] is not None:
    print("Beraterin zugeteilt!")

### Aufgabe: Hill climbing

Implementieren Sie die Methode <code>tweak</code> um den hill climbing-Algorithmus zu vervollständigen. <code>tweak</code> soll
* eine zufällige Beraterin (B1) auswählen -- hier ist die Methode [random.randrange](https://docs.python.org/3/library/random.html#functions-for-integers) nützlich
* ein zufälliges Projekt (P2) auswählen
  
Weiterhin sei P1 das aktuelle Projekt von B1 (sofern vorhanden) und B2 die aktuelle Beraterin von Projekt P2 (sofern vorhanden). Nun sollen B1 und B2 die Projekte tauschen. Wenn B2 nicht existiert, so wechselt B1 einfach das Projekt (P1 zu P2). Wenn P1 nicht existiert, so wechselt P2 einfach die Beraterin (von B2 zu B1). Achten Sie darauf sowohl <code>consultants_mapping</code> als auch <code>projects_mapping</code> zu aktualisieren.

In [ ]:
def tweak(consultants_mapping,projects_mapping):
    ### Ihr Code hier ###

    ### Ende Ihres Codes ###

def get_assignment(consultants_mapping):
    return [(c,p) for c,p in enumerate(consultants_mapping) if p is not None]

def __compute_profit(consultants_mapping,profits):
    return sum([profits[c,p] for c,p in enumerate(consultants_mapping) if p is not None])

def hill_climbing(assignment,profits,time_budget):
    start_time=time.time()
    consultants_mapping,projects_mapping=get_mappings(assignment,profits)
    current_profit=__compute_profit(consultants_mapping,profits)
    while time.time()-start_time<time_budget:
        new_consultants_mapping=consultants_mapping.copy() # wir machen eine Kopie, damit wir die Änderung später nicht rückgängig machen müssen
        new_projects_mapping=projects_mapping.copy() # wir machen eine Kopie, damit wir die Änderung später nicht rückgängig machen müssen
        tweak(new_consultants_mapping,new_projects_mapping)   
        new_profit=__compute_profit(new_consultants_mapping,profits)  
        if new_profit>current_profit:    # we found an improvement, so we take the step 
            consultants_mapping=new_consultants_mapping
            projects_mapping=new_projects_mapping
            current_profit=new_profit
    assignment=get_assignment(consultants_mapping)
    return current_profit,assignment 

profits=random_instance(10,12,0.3,seed=42)

profit,assignment=random_search(profits,1)
profit,assignment=hill_climbing(assignment,profits,1)
print("Realisierter Profit: {}k€".format(profit))

## Vergleich

Wir wollen wieder die verschiedenen Algorithmen halbwegs systematisch vergleichen. Erzeugen Sie mit der Methode <code>random_instance(50,60,0.5)</code> 50 Zufallsinstanzen mit jeweils 50 Mitarbeitern, 60 Projekten und einer edge_proba von 0.5. Vergleichen Sie für die 50 Zufallsinstanzen die drei Algorithmen *greedy*, *hill climbing* und *Zufallssuche*. Merken Sie sich jedes Mal, wie viel Profit erzielt wurde, und welcher Algorithmus am besten ist. Als Zeitbudget sollen *hill climbing* und *Zufallssuche* jeweils 1s erhalten (das ist wenig, führt aber schon zu einer Laufzeit von fast zwei Minuten). Starten Sie *hill climbing* mit der *greedy*-Lösung.

In [ ]:
greedy_profits = []
hill_profits = []
random_profits = []
wins = [0]*3
time_budget=1
repeats = 50

### Ihr Code hier ###

### Ende Ihres Codes ###

print("Bestes Ergebnis in {} Versuchen:".format(repeats))
print("Greedy            : {:2.1f}%".format(wins[0]/repeats*100))
print("Hill climbing     : {:2.1f}%".format(wins[1]/repeats*100))
print("Zufallssuche      : {:2.1f}%".format(wins[2]/repeats*100))
print('---------------------------------')
print("Durchschnittlicher Profit")
print("Greedy            : {:.1f}k€".format(sum(greedy_profits)/repeats))
print("hill climbing     : {:.1f}k€".format(sum(hill_profits)/repeats))
print("Zufallssuche      : {:.1f}k€".format(sum(random_profits)/repeats))